# Décodage du charabia avec linguistica
Produire trois version du texte avec :
- la phrase d'origine
- la phrase découpée
- la glose pour chaque morphème

À terme produire un document LaTeX avec en plus une ligne pour noter la traduction

In [1]:
import linguistica as lxa
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import re,tabulate,textwrap
from itertools import combinations
lxa.__version__  # show version number

/Users/gilles/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'5.2.1'

## Lire le corpus

In [2]:
charabia="Aveugle"
repName="/Users/gilles/ownCloud/Cours/Bordeaux/M2-TraitementsCorpus/"
fName="Corpus%s-Sentenced-Ordered.txt"%charabia
# lxaCorpus = lxa.read_corpus(repName+fName)

In [3]:
texteBrut=pd.read_csv(repName+fName,encoding="utf8",header=None,sep="\t").values.tolist()
texteBrut=[t[0] for t in texteBrut]
compW=re.compile(u"(\W)+",re.U)
texteBrut=[re.sub(compW," ",l) for l in texteBrut]
lxaCorpus = lxa.from_corpus(texteBrut)

In [4]:
%store -r conteLines

lineConte={}
nContes=[]
nConte=""
nDebutConte=0
for nLine,line in enumerate(texteBrut):
    m=re.search(u"^(\d+)",line)
    if m:
        nConte=m.group(1)
        nContes.append(nConte)
    lineConte[nLine]=nConte
# lineConte

In [5]:
 tradContes=set(nContes).intersection(set(conteLines))

## Basic information

In [6]:
print ("mots-occurrences",lxaCorpus.number_of_word_tokens())
print ("mots-types",lxaCorpus.number_of_word_types())

mots-occurrences 206136
mots-types 8720


## Séquences fréquentes
Liste des séquences de mots au dessus du *seuil* de fréquence pour :
- les mots seuls
- les bigrammes
- les trigrammes

In [7]:
def spreadListe(liste,cols):
    result=[]
    if isinstance(liste[0][0],tuple):
        locListe=[" ".join(seq)+" : "+str(nb) for seq,nb in liste]
    else:
        locListe=[seq+" : "+str(nb) for seq,nb in liste]
#     print locListe
    pos=[None for c in range(cols)]
    for n,l in enumerate(locListe):
        i=n%cols
        pos[i]=l
        if i+1==cols:
            result.append(pos)
            pos=[None for c in range(cols)]
    print (tabulate.tabulate(result))
    return 

In [8]:
seuilUnigram=1000
dUnigram=lxaCorpus.word_unigram_counter()
pUnigram=sorted(dUnigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print ("mots de fréquence supérieure à %d"%seuilUnigram)
freqUnigrams=[]
for w,f in pUnigram:
    if f>seuilUnigram:
#         print w,f
        freqUnigrams.append((w,f))
    else:
        break
spreadListe(freqUnigrams,6)

mots de fréquence supérieure à 1000
----------  ------------  ----------  ------------  -----------  ----------
ed : 10267  ñax : 5018    o : 4114    zix : 4051    zih : 4028   ah : 3784
ix : 3553   hoy : 3434    wta : 3101  tid : 2670    ossi : 2623  raq : 2604
zi : 2588   ñux : 2575    vta : 2554  wadzi : 2396  ziq : 2331   za : 2210
ap : 2035   cix : 1941    zu : 1935   ciz : 1836    wod : 1763   iq : 1738
peq : 1590  ñarzi : 1466  ñac : 1439  sec : 1392    xu : 1374    ih : 1320
----------  ------------  ----------  ------------  -----------  ----------


In [9]:
seuilBigram=200
dBigram=lxaCorpus.word_bigram_counter()
pBigram=sorted(dBigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print ("séquences de deux mots de fréquence supérieure à %d"%seuilBigram)
freqBigram=[]
for w,f in pBigram:
    if f>seuilBigram:
#         print " ".join(w),f
        freqBigram.append((w,f))
    else:
        break
spreadListe(freqBigram,4)

séquences de deux mots de fréquence supérieure à 200
---------------  --------------  -------------  ---------------
hoy ah : 525     ed wadzi : 516  za ñax : 494   ed za : 469
peq ah : 454     zih iq : 430    zih raq : 399  wadzi ñax : 398
ziq raq : 392    ah ñax : 368    oxz o : 358    zix dacyi : 299
ñax ñarzi : 291  o ix : 258      za zi : 252    ed wta : 250
tid ñaq : 249    ap whiz : 243   za ñux : 233   ed dos : 227
ed zi : 223      ñix hoy : 222   ev ed : 214    hoy ix : 212
---------------  --------------  -------------  ---------------


In [10]:
seuilTrigram=50
dTrigram=lxaCorpus.word_trigram_counter()
pTrigram=sorted(dTrigram.items(), key=lambda kv: (kv[1], kv[0]), reverse=True)
print ("séquences de trois mots de fréquence supérieure à %d"%seuilTrigram)
freqTrigram=[]
for w,f in pTrigram:
    if f>seuilTrigram:
#         print " ".join(w),f
        freqTrigram.append((w,f))
    else:
        break
spreadListe(freqTrigram,4)

séquences de trois mots de fréquence supérieure à 50
-----------------  --------------------  -----------------  --------------------
ed za ñax : 151    ziq raq ixdaxd : 118  za ñax sec : 109   raq ixdaxd ix : 107
wod hoy ah : 94    riz wozix ap : 85     ed za ñux : 84     ah zih raq : 79
uz o wserix : 71   za ñax ñarzi : 68     yer hoy ah : 66    o ix paqh : 65
ed za zi : 62      za zi ñarzi : 59      o wserix ed : 58   ah ñax wsuyyi : 58
ed wahhi wod : 56  ed wadzi hoy : 56     za ñax wta : 55    wadzi ñax hoy : 55
za zi sec : 54     za ñax raq : 53       ykwh hoy ah : 53   wadzi zix dacyi : 53
ed hoy wozwh : 53  cix za ñax : 53       tid wsay xes : 51  ed wadzi ah : 51
-----------------  --------------------  -----------------  --------------------


# Découpage

In [11]:
dictBrokenWords=lxaCorpus.broken_words_left_to_right()
dictGlossWords={k:"-".join(v) for k,v in dictBrokenWords.items()}
dictHyphenWords={k:"-".join(v) for k,v in dictBrokenWords.items()}

In [12]:
dictFoundGloss={
    "zesheq":"docteur[nom1]",
    "zeshiq":"docteur[nom2]",
    "zesheqix":"docteur-DEF",
    "zad":u"jour",
    "teti":u"œil",
    "ixteti":u"1-œil",
    "heteti":u"2-œil",
    "hqiteti":u"3-œil",
    u"ñocyix":"ciel-DEF",
    "taxzix":"esprit-DEF",
    "wyaxdi":"serpent",
    "pyawsix":"bouteille-DEF",
    "ayrozixzi":"universel[adj]",
    "jexzix":"paysan-DEF",
    "jexzi":"paysan",
    "vixdi":"seigneur?",
    "vixdixi":"seigneur?",
    "wtex":"fils",
    "wtexxiq":"fils-PL",
    "ytexxi-h":"fils?",
    "zahhiq":"fille",
    "vodix":"jeune.fille-DEF",
    "jqtezqi":u"frère",
    "jqeqyoy":u"frérot",
    "wtewhiqyoy":u"sœurette",
    "zqixdi":u"garçon",
    "paq":u"père",
    "jaq":u"enfant",
    "jaqx":u"enfant-?",
    "caxz":"homme",
    "caxzix":"homme-DEF",
    "caxzixw":"homme-DEF-POSS",
    "sexi":"femme",
    "sexix":"femme-DEF",
    "pudy":"oiseau",
    "pudyix":"oiseau-DEF",
    "udyix":"chouette-DEF",
    "qarxi":"corbeau-PL?",
    "pqte":"grenouille",
    "pqteix":"grenouille-DEF",
    "dtiwwixi":"oie-DEF-PL",
    "sah":"chat",
    "sahhix":"chat-DEF",
    "cuw":"souris",    
    "cuwix":"souris-DEF",
    "diz":u"chèvre",
    "dizix":u"chèvre-DEF",
    "dizisoz":u"chevreau-PL",
    "dizisozzixi":u"chèvre-PETIT-DEF-PL",
    "uyrix":u"loup-DEF",
    "jtteqxix":u"ours-DEF",
    "tiwiy":u"âne",
    "qtirix":u"renard-DEF",
    "yuwix":u"pou-DEF",
    "yevvix":u"puce-DEF",
    "wser":u"forêt",
    "wserix":u"forêt-DEF",
    "sudyi":u"balle",
    "sudyix":u"balle-DEF",
    "sqexi":u"couronne",
    "wuyhax":"sultan",
    "sexdi":"roi",
    "sexdix":"roi-DEF",
    "zttiriyix":"diable-DEF",
    "zttiriyixw":"diable-DEF-GEN",
    "duyz":"or",
    "cizowhiqvteywi":"saucisse",
    "cizowhiqvteywix":"saucisse",
    "zahhiqix":u"château-DEF",
    "qtezñtihhi":"rouge+chaperon",
    "wyaxdijyazi":"serpent+feuille",
    "pqtesexdix":"grenouille+roi-DEF",
    "sexdizahhiqix":u"roi+château-DEF",
    "sexdiwtexxixw":"roi+fils?",
    "wtexxiwtexxix":"fils+fils?",
    "dyawsowhix":u"verre+cercueil-DEF",
    "duyzsudyi":"or+balle",
    "duyzsqexi":"or+couronne",
    "piqzoxaxz":"Fernand",
    "awsiveh":u"Cendrillon",
    "iywi":u"Élise",
    "qavuxgiy":u"Raiponce",
    "teqoxzi":u"Jorinde",
    "teqoxdiy":u"Joringel",
    u"wxiñrozi":u"Blanche-Neige",
    "sowhiq":"cercueil",
    "jqtexzix":"fontaine-DEF",
    "ctaxix":"lune",
    "wvoyyicaxz":"violon",
    "wixdi":"lit",
    "jtidiq":"gobelet",
    "yadixiq":"drap-PL",
    "cuxz":"bouche",
    "cuxzix":"bouche-DEF",
    "wvqed":"langue",
    "wsqtizziq":"tailleur",
    "wsqtizziqix":"tailleur-DEF",
    "powsiqix":u"pêcheur-DEF",
    u"jqtixziñuddiq":u"bûcheron(x+y?)",
    u"jqtixziñuddiqix":u"bûcheron(x+y?)-DEF",
    "ciwhiq":"patron",
    "soqsidtaqzix":u"cimetière-DEF",
    "soqsi":u"église",
    "soqsix":u"église",
    "wteccih":"clou-DEF",
    "ptiq":"plume-PL",
    "jtexxix":u"haricot-DEF",
    "dxowhix":u"braise-DEF",
    "ptiyzi":u"piège",
    "daxd":"temps",
    "taq":"an",
    "ctiys":"lait",
    "xowwiqxi":"lutin-PL",
    "raxz":"eau",
    "zixdaxd":"temps[DEF.PL]",
    "ixdaxd":"temps[INDEF?]",
    u"ñtixziq":"main",
    u"ñrozi":"blanc",
    u"jyta":"bleu?",
    "syed":"sage[SG]",
    "syedi":"sage-i[PL]",
    "syedh":"sage-h",
    "syedix":"sage-DEF",
    "yoyyi":"petit",
    "wcta":"petit?",
    "uxziqyod":"merveilleux",
    "uxziqyodi":"merveilleux-i",
    "uxziqyodh":"merveilleux-h",
    "rozuxziqyod":"merveilleux",
    "rozuxziqyodi":"merveilleux-i",
    "ykssiyod":"ensemble",
    "ykssiyodh":"ensemble",
    "ykssiyodi":"ensemble",
    "uhasxicyodi":"ingrat",
    "ixiwhi":u"seul/unique?",    
    "whoyyi":u"calme/tranquille?",    
    "whoyyizi":u"calme/tranquille?",    
    "whoyyiw":u"calme/tranquille?",    
    "whoyyih":u"calme/tranquille?",   
    "hqe":u"loyal",
    "uhqe":u"dé-loyal",
    "wacci":u"bien?",
    "waccix":u"bien?",
    "ayhwacci":u"pas-bien?",
    "ayyiwaccix":u"?pas-bien?",
    "dezi":u"bon[??]",
    "wheq":u"grand[??]",
    "cteqs":u"sombre[??]",
    "dacyi":u"vieux[??]",
    "dacyiw":u"vieux[??]",
    "wtezi":u"doux[??]",
    "exzi":u"mauvais[??]",
    "pyoxs":u"intelligent/avisé[SG]",
    "pyoxsi":u"intelligent/avisé[PL?]",
    "pahhodi":u"pauvre",
    "zityod":u"beau[??]",
    "payziqa":u"tombé[??]",
    "payzi":u"tombé[??]",
    "payziq":u"tombé[??]",
    "payzih":u"tombé[??]",
    "payzh":u"tombé[??]",
    "wvuqdhi":u"demander/répondre?-hi[PST]",
    "wvowi":"dormir",
    "hewwih":"dire???-hi[PST]",
    "sayzh":"s'appeler?-h",
    "rawsi":"se laver[FUT]",
    "yirizi":"vivre-zi[PST.3PL?]",
    "jyirih":"vivre-ih[PST.3PL?]",
    "zqossi":"boire[FUT]",
    "ossi":"pouvoir",
    u"ñreq":u"PRO.3SG-être[PRS]",
    u"ñric":u"PRO.3SG-être[PST?]",
    u"ñixzi":u"être[PST?]",
    "iq":u"être-COP[PRS.2SG]",
    u"raq":u"être-COP[PST]",
    u"ñraz":u"PRO.3SG-être/avoir[?]",
    u"ñarzi":"PRO.3SG-avoir[PST?]",
    u"wadzi":"dire[PST?]",
    u"ztezi":"mourir[PST]",
    u"ztezix":"mort-DEF",
    u"ztezixw":"mort-DEF-GEN",
    u"ñax":"PRO.3SG",
    u"ñaxw":"PRO.3SG-POSS",
    "cax":"PRO.1SG",
    "zu":"PRO.2SG",
    "zi":"PRO.3PL?/DEF.PL",
    "ziq":"PRO.3SG",
    "ziqiw":"PRO.3SG-POSS",
    "dqihi":u"PRO?/Gretel?",        
    u"ñrow":"DONT/POSS?",
    u"cox":"POSS.1SG",
    u"coxi":"POSS.1SG-PL",
    u"zox":"POSS.2SG",
    u"zoh":"POSS.2SG",
    u"wox":"POSS.3SG",
    u"woxi":"POSS.3SG-PL",
    "zix":"DEF[SG.?]",
    "zih":"DEF[SG.?]",
    "ed":"ET",
    "edwta":"ET-AUSSI",
    "ix":"1-INDEF",
    "ih":"INDEF",
    "he":"2",
    "hqi":"3",
    "tiyzwhi":u"premier",
    "kxdwhi":u"second",
    "hqizoi":u"troisième",
    "wkr":"7",
    "wisw":"6?",
    "heyr":"12",
    "zadi":u"TRÈS",
    "o":"DANS",
    "wec":"SUB",
    "cix":"MAIS",
    "ciz":"AVEC",
    "uzix":"SANS",
    "pqa":"DE(GEN)",
    "dos":"DE(PROV)",
    "wta":"COMME/AUSSI",
    "wyih":"NEG",    
#    "ah":"CONJ.QUE",
    }

In [13]:
for k,v in dictFoundGloss.items():
    dictGlossWords[k]=k+"@"+v

In [14]:
texteHyphen=[]
texteGloss=[]
for nLineOrig,lineOrig in enumerate(texteBrut):
    lineHyphen=[]
    lineGloss=[]
    print (nLineOrig+1)
    for word in lineOrig.split(" "):
        if word:
            lineHyphen.append(dictHyphenWords[word])
            lineGloss.append(dictGlossWords[word])
    texteHyphen.append(" ".join(lineHyphen))
#    if nLineOrig+1==108:
#        print lineGloss
    texteGloss.append(" ".join(lineGloss))
#    print texteHyphen[-1]
#    print
#    if texteHyphen[-1]!=texteGloss[-1]:
    print (textwrap.fill(texteGloss[-1],width=80,subsequent_indent='  '))
    print

1
001 pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos
2
o@DANS dacyi@vieux[??] zadi@TRÈS zixdaxd@temps[DEF.PL] cax@PRO.1SG suxxi pta
  woxi@POSS.3SG-PL texwsiq evpkyzh yirizi@vivre-zi[PST.3PL?] ziq@PRO.3SG
  ix@1-INDEF sexdi@roi wec@SUB ñarzi@PRO.3SG-avoir[PST?] ix@1-INDEF zahhiq@fille
  ziq@PRO.3SG raq@être-COP[PST] wta@COMME/AUSSI zityod@beau[??] ah wiyr weyix
  ziq@PRO.3SG zed ñaq wih wta@COMME/AUSSI cidih stexh uxzqi-zi-w ñriq daxd@temps
  zix@DEF[SG.?] wsoxxizi vta ñixzi@être[PST?]
3
o@DANS xtiqñizix ap wyehhih yta ziq@PRO.3SG ix@1-INDEF wheq@grand[??]
  cteqs@sombre[??] wser@forêt ed@ET ziq@PRO.3SG oxzi raq@être-COP[PST]
  ziq@PRO.3SG ix@1-INDEF jqtexz uxziq ix@1-INDEF dacciy yoxz
4
xtaq zih@DEF[SG.?] raq@être-COP[PST] cidih raqch dos@DE(PROV)
  sexdizahhiqix@roi+château-DEF uz o@DANS wserix@forêt-DEF ed@ET wahhi wod riz
  raxzih ed@ET xtaq ñux sizi-zi wod wvoyyizi ñux jeyz ciz@AVEC ix@1-INDEF
  duyzsudyi@or+balle wec@SUB raq@être-COP[PST] ñixziw stiqiwhi yidihtet
5
ix@1-INDEF zad

tid iq@être-COP[PRS.2SG] ytejiq wraqizi ñax@PRO.3SG ed@ET tid ñaq hadih jixih ap
  peq ah zih@DEF[SG.?] ossi@pouvoir wsay dta ayh peq ñuqhodh
5885
xtaq tid ytejiq vta jiddi jix dtaq zih@DEF[SG.?] ñuqhodiqi ixz ix@1-INDEF
  pudy@oiseau sax pykri
5886
roy zu@PRO.2SG ossi@pouvoir dta ciz@AVEC cod
5887
wvuqdhi@demander/répondre?-hi[PST] caxzix@homme-DEF ro pic wsay xes secci dixxic
  ñiyi riqzix
5888
zi@PRO.3PL?/DEF.PL dos@DE(PROV) xu ayyiwaccix@?pas-bien? roziqi ed@ET yozh iphiq
  ctezhi zi@PRO.3PL?/DEF.PL ix@1-INDEF pkq wec@SUB ñarzi@PRO.3SG-avoir[PST?]
  ñahhix vta zih@DEF[SG.?] ixi teqi
5889
ñreqzax iq@être-COP[PRS.2SG] zih@DEF[SG.?] zed zu@PRO.2SG wiq uz
  wadzi@dire[PST?] caxzix@homme-DEF zu@PRO.2SG yodxiq te ix@1-INDEF xaq
5890
wtih zed zox@POSS.2SG ñah eqzixhyodh
5891
zih@DEF[SG.?] hteq tid ossi@pouvoir wraqizi ñax@PRO.3SG peq xtaq tid dteq
  zih@DEF[SG.?] jyoriq zih@DEF[SG.?] wta@COMME/AUSSI seyzh ah pudyixi pqkwiq
  oñtiy ed@ET payziq@tombé[??] ztezi@mourir[PST] hoy teqzix
5892
s

zih@DEF[SG.?] ctahhi tid wraqizi zttiriyix@diable-DEF tid suxxi sux wsaji uz
  pqa@DE(GEN) cox@POSS.1SG idix xahuq wec@SUB iq@être-COP[PRS.2SG] ah dteqi
  zih@DEF[SG.?] exzi@mauvais[??]
10793
zu@PRO.2SG wsay iqwhah-hi cod zih@DEF[SG.?]
10794
xtaq idix hajiq woh yter wsay tid jihayi zod wadzi@dire[PST?] reqñiqqi tid ñaq
  hayh vixdixi@seigneur? ap hoy zod
10795
za idijyazixi yta wvqizh vta teqzix sec zttiriyix@diable-DEF ed@ET peqyaxdhi
  woxi@POSS.3SG-PL vixdi@seigneur?
10796
o@DANS ix@1-INDEF soqsi@église o@DANS sexwhaxhoxeviy whtaq ziq@PRO.3SG ixzxu
  ix@1-INDEF id wec@SUB ñaq ayyi woxi@POSS.3SG-PL jyazi wraqizi duz
10797
qawixzi peq zttiriyix@diable-DEF ap whiz peq ah poxzi idix cix@MAIS raxzq-izi
  o@DANS wisw@6? ctaxi-ziq ec o@DANS teqsixix ed@ET za ñax@PRO.3SG sec ñtic
  raq@être-COP[PST] ayyi hqtiiq dqtexxi
10798
ñax@PRO.3SG ctahhi za apwhta pqa@DE(GEN) wox@POSS.3SG peqz-q-o-xd cix@MAIS
  o@DANS qawiqo whas ñax@PRO.3SG tetxixi uz vta ayyi zi@PRO.3PL?/DEF.PL diziq
  ñax@PRO.3SG ñ

## Concordances

### Texte brut

In [74]:
def displayConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    lLines=[]
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    for nLine,line in enumerate(texteBrut[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite
                
                recLine=[{"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}]
                dfLine=pd.DataFrame.from_records(recLine)
                lLines.append(dfLine)

    dfConcordance=pd.concat(lLines,ignore_index=True)

    # dfStyler=dfConcordance.style.set_properties(subset=[concordance],**{'font-weight': 'bold'})
    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'})
    display(dfStyler)
    return


In [73]:
displayConcordance(" whoyyi")

,conte,num,gauche,whoyyi,droite
0,001,69,hizi xu sexdiwtexxix ed vqoxwiwwix oxz o redxix ed,whoyyi,-zi wod jadriz zic ed ñaxw ñtiqhi wruycizi ap dytiz
1,002,84,shi wod ytixdi ec ed ixziyod wadzi zix ro sax ossi,whoyyi,zix xedih wosqiqi whiz ixz o soqsix ziq iq ziq ox
2,003,163,ñux har wta,whoyyi,cix xkwdiqqodñizix yez ñixzi ossi ñari pqiz
3,003,216,rzi tizh zih ed coxowhqixi peqyaxdhi ah ñux wsuyyi,whoyyi,-w peq ix zecwhey
4,003,225,ñux jyir,whoyyi,-h peq ix zecwhey ed za ñux ossi suxxi peqwraqi wod
5,004,234,x kxdwhi raq wta zuc ah ziq wyih ossi raq xedih ah,whoyyi,ev ciz ñac
6,004,248,c ed wadzi ah ñax wyih ossi rozwhi ñraz ñax wsuyyi,whoyyi,ev ciz wox kxdwhi wtex ñax raq wta zuc wta zih os
7,004,263,xshi zidxix ed dar ossi ix ykz pqa wod ed whez wta,whoyyi,wec ec ñax raq ap whix
8,004,324,ho zed,whoyyi,wadzi ñaxw sexi ziq iq ayyiqizi wta caxdi ziq ñaq
9,004,337,hixix dos zqixdix ziqoxz htixzhi wod ih syaqh jtay,whoyyi,-zi ñteryijtixsix ciz sxorix riz wozix ap ed wahhi


In [75]:
displayConcordance(" wvtiqqi")

,conte,num,gauche,wvtiqqi,droite
0,011,988,x za qtazkqih wadzi tid zteq ap weqd ñrow tid wsay,wvtiqqi,-w oxzi ñiq suxxi ñux ossi axzih ixz ciz huxdh ñtiq
1,019,1716,ñax hqeizi ñax ñarzi ñteqh dayh ed,wvtiqqi,-zi tetxixi rozh ev ed wadzi ñraz iq zih zed zu wod
2,057,4693,ax ptiqxi zih jtiqd ziq yoddiq uzixpeq coh wyeh ed,wvtiqqi,-q uzwodhix
3,060,5163,ap ritix peq zix cix za zix sec hoy wsoyzradhiqxi,wvtiqqi,-zi zi ritix peq zix ciz ziqiw jtewwiq ed royyi oss
4,060,5173,wsoyzradhiqxi royyi,wvtiqqi,ritix peq zix cix zix uzwhtezhi jyeh ih jqtey wta
5,071,5913,hui ñreq duyrih ed zteqix raq ap tiqx ed roxzuiqxi,wvtiqqi,-h ciz tiqxwhtixdiq
6,166,11395,zi dos xu uz ap ñuyix ed zih sax xes rtiqi ah ñaxw,wvtiqqi,-zi tetxixi ev za ñax pqa cteqsih sec uz o zih syaq
7,166,11459,i ed za ñux ossi royyi dophi wod ciz ñac ñarzi ñax,wvtiqqi,-h ñixzi oxzi o syovvix ed wah zix exzwsajwpuyzi zr
8,169,11625,zi iq,wvtiqqi,-h oxzi o stiyziqix wraqizi ñax o ceqdix wsay zi pt
9,172,11710,wh azwsoyhi zic ziq diqxi royyi rtiqi waccix iyyiq,wvtiqqi,-zi ritix peq zic


In [35]:
conte="031"
if conte in tradContes:
    for lConte in conteLines[conte]:
        print (textwrap.fill(lConte,width=80,subsequent_indent='  '))

031 la jeune fille sans mains
il était une fois il y a quelques jours à l époque où la farine des villageois
  était écrasée à la meule de pierre un meunier qui avait connu des temps
  difficiles
il ne lui restait plus que cette grosse meule de pierre dans une remise et
  derrière un superbe pommier en fleur
un jour tandis qu il allait dans la forêt couper du bois mort avec sa hache au
  tranchant d argent un curieux vieillard surgit de derrière un arbre
a quoi bon te fatiguer à fendre du bois
dit il
ecoute si tu me donnes ce qu il y a derrière ton moulin je te ferai riche
qu y a t il derrière mon moulin sinon mon pommier en fleurs
pensa le meunier
il accepta donc le marché du vieil homme
dans trois ans je viendrai chercher mon bien gloussa l étranger avant de
  disparaître en boitant derrière les arbres
sur le sentier en revenant le meunier vit son épouse qui volait à sa rencontre
  les cheveux défaits le tablier en bataille
mon époux mon époux quand l heure a sonné une pendule magnif

### Texte glosé

In [76]:
def displayGlossConcordance(concordance,fenetre=50):
    blancGauche=concordance.startswith(" ")
    blancDroite=concordance.endswith(" ")

    dfConcordance=pd.DataFrame(columns=["conte","num","gauche",concordance,"droite"])
    p=re.compile("(.*)(%s)(.*)"%concordance,re.U)
    lLines=[]
    for nLine,line in enumerate(texteGloss[:]):
        occurrences=re.findall(p,line)
        if occurrences:
            for occurrence in occurrences:
                if len(occurrence[0])>fenetre:
                    gauche=occurrence[0][-fenetre:]
                else:
                    gauche=occurrence[0]
                if not blancGauche and gauche and gauche[-1]!=" ":
                    gauche=gauche+"-"

                if len(occurrence[2])>fenetre:
                    droite=occurrence[2][:fenetre]
                else:
                    droite=occurrence[2]
                if not blancDroite and droite and droite[0]!=" ":
                    droite="-"+droite

                recLine=[{"conte":lineConte[nLine],"num":nLine+1,"gauche":gauche,concordance:occurrence[1],"droite":droite}]
                dfLine=pd.DataFrame.from_records(recLine)
                lLines.append(dfLine)

    dfConcordance=pd.concat(lLines,ignore_index=True)

    dfStyler=dfConcordance.style.set_properties(subset=['gauche'],**{'text-align': 'right'})
    dfStyler=dfConcordance.style.set_properties(subset=['droite'],**{'text-align': 'left'})
    dfStyler.hide(axis="index")
    display(dfStyler)
    return

In [77]:
displayGlossConcordance(u" viqy",50)

conte,num,gauche,viqy,droite
001,14,teau-DEF coxi@POSS.1SG-PL sytiziq coxi@POSS.1SG-PL,viqy,-iq ed@ET coxi@POSS.1SG-PL tizi-ywhixi ed@ET ñrow@D
015,1392,ziq@PRO.3SG whez wheqi sawwi-q puyzi ap,viqy,-iq ed@ET tizi-ywhixi
015,1410,dqihi@PRO?/Gretel? sawhizi,viqy,-i-qxi ed@ET tizi-ywhixi uz ap peqsytizi-h ed@ET ña
021,1901,wcussi sytiziq wadzi@dire[PST?] zix@DEF[SG.?] ixi,viqy,-iq ed@ET tizi-ywhixi jaz zix@DEF[SG.?] axzix
021,1904,PRO.3SG stejhi za edwta@ET-AUSSI xki sytiziq ed@ET,viqy,-iq ed@ET tizi-ywhixi hoy woxi@POSS.3SG-PL he@2 zte
021,1939,@or ed@ET wteyr-oxzroqsih steyi ed@ET ih@INDEF vaq,viqy,-ijqeziqizi wse xiz hoy ñixzi@être[PST?]
039,3353,-INDEF wixd ap weqh ojixhqti ciz@AVEC sxav-vi-q ap,viqy,-iq htivviqxi raq@être-COP[PST] duyzj-qeziqizi rudd
062,5442,whi wsuyyi cax@PRO.1SG wacyi huwoxz ap vqoxwiwwixw,viqy,-iq wec@SUB yta uzi uxziq cewwih o@DANS wserix@forê
062,5444,uvoir wheqh jizqi ñax@PRO.3SG paxzh sux heñuxzqizi,viqy,-iq ed@ET jyir edwta@ET-AUSSI hoy whix
062,5447,a@petit? zkq ñarzi@PRO.3SG-avoir[PST?] puxzih ayyi,viqy,-i-qxi ed@ET jtaqih zic waccix@bien? o@DANS ix@1-IN


In [78]:
displayGlossConcordance(u"^\d\d\d ",80)

conte,num,gauche,^\d\d\d,droite
001,1,,001,pqtesexdix@grenouille+roi-DEF iyyiq tiqxñixqos
002,80,,002,sah@chat ed@ET cuw@souris o@DANS jeptiyyiwsaj
003,140,,003,caqoijaqxih
004,232,,004,irixhkq-ih ec ix@1-INDEF ziq@PRO.3SG zqed uz peq ah ytiqi pqkdh ah sixzi
005,445,,005,uyrix@loup-DEF ed@ET zi@PRO.3PL?/DEF.PL wkr@7 dizisoz@chevreau-PL
006,505,,006,zix@DEF[SG.?] hqe@loyal teñaxxiw
007,654,,007,zix@DEF[SG.?] dezi@bon[??] ñaxziy
008,744,,008,zix@DEF[SG.?] uxziqyodi@merveilleux-i wvoyyicaxz@violon
009,795,,009,zi@PRO.3PL?/DEF.PL heyr@12 jqtezqi@frère
010,900,,010,vas


## Calculer une hiérarchie de signatures morphologiques
Les signatures morphologiques de Linguistica correspondent à un paradigme flexionnel réduit aux variantes attestées dans le corpus pour un même lexème. Hiérarchiser les signatures permet d'étudier les regroupements possibles entre paradigmes partiels.

In [79]:
morphGraph=nx.Graph()
sigGraph=nx.DiGraph()

In [80]:
for sig in lxaCorpus.signatures():
    for a,b in combinations(sig,2):
        if (a,b) not in morphGraph.edges:
            morphGraph.add_edge(a,b,weight=1)
        else:
            morphGraph[a][b]["weight"]+=1

### Cliques maximales parmi les signatures

In [81]:
maxCliques=list(nx.find_cliques(morphGraph))
# maxCliques

In [82]:
dCliques={tuple(c):set() for c in maxCliques}
for sig in lxaCorpus.signatures():
    sSig=set(sig)
    for clique in dCliques:
        sClique=set(clique)
        if sSig.intersection(sClique)==sSig:
            dCliques[clique].add(sig)
print ("Les cliques maximales et leurs sous-cliques sont :")
print
for cMax,lCliques in dCliques.items():
    if len(lCliques)>1:
        print (cMax,":")
        print ("\t",list(lCliques))
    else:
        print (cMax)
    print

Les cliques maximales et leurs sous-cliques sont :
('oxd', 'i')
('hi', 'h', 'i') :
	 [('h', 'hi', 'i'), ('h', 'i')]
('NULL', 'zi', 'xzi', 'h', 'q', 'w') :
	 [('NULL', 'q', 'xzi'), ('h', 'zi'), ('NULL', 'h', 'w'), ('NULL', 'q'), ('q', 'zi'), ('NULL', 'q', 'w'), ('NULL', 'zi'), ('NULL', 'h', 'q', 'w', 'zi'), ('NULL', 'h', 'q'), ('NULL', 'q', 'w', 'xzi'), ('q', 'xzi', 'zi'), ('NULL', 'h', 'zi'), ('NULL', 'h', 'q', 'xzi', 'zi'), ('NULL', 'q', 'zi'), ('xzi', 'zi'), ('NULL', 'xzi'), ('NULL', 'h', 'q', 'zi'), ('NULL', 'h'), ('NULL', 'w'), ('h', 'q'), ('h', 'q', 'zi')]
('NULL', 'zi', 'xzi', 'h', 'q', 'x') :
	 [('NULL', 'x'), ('NULL', 'q', 'xzi'), ('h', 'zi'), ('NULL', 'q'), ('NULL', 'x', 'zi'), ('q', 'zi'), ('NULL', 'h', 'x'), ('NULL', 'zi'), ('NULL', 'q', 'x'), ('NULL', 'h', 'q'), ('NULL', 'h', 'x', 'zi'), ('NULL', 'q', 'x', 'zi'), ('q', 'xzi', 'zi'), ('NULL', 'h', 'zi'), ('NULL', 'h', 'q', 'xzi', 'zi'), ('NULL', 'q', 'zi'), ('xzi', 'zi'), ('NULL', 'h', 'q', 'x', 'zi'), ('NULL', 'xzi'), ('NUL

In [83]:
sigMax={}
for k,l in dCliques.items():
    for e in l:
        if e not in sigMax:
            sigMax[e]=set()
        sigMax[e].add(k)
sigMax[(u'NULL', u'ix')]

{('NULL', 'i', 'ix', 'iq'),
 ('NULL', 'i', 'ix', 'ixi', 'ixw'),
 ('NULL', 'i', 'ix', 'w'),
 ('NULL', 'iqxi', 'iq', 'ix')}

In [84]:
lxaCorpus.stems()
lxaCorpus.affixes()
dSignatureStems=lxaCorpus.signatures_to_stems()

In [85]:
keySignatures=[]
for k in lxaCorpus.signatures_to_stems().keys():
    if len(k)>4:
        keySignatures.append(k)

In [86]:
for k in keySignatures:
    print (", ".join(k))
    print ("\t",", ".join(lxaCorpus.signatures_to_stems()[k]))
    print

NULL, i, ix, ixi, ixw
	 ñisw, jtteqx, zqixd, ñiwh, wrixz
NULL, h, q, x, zi
	 syovvi, ñroyi, reswi, ñassi, qidxi, vqteri, paxdi, yazi, zaxwi, rtadi
NULL, h, q, w, zi
	 tadi, whoyyi, ñuddi, ñixhi, pitqi, wvtiqqi
NULL, i, ih, iq, izi
	 ñitw, texws, yteph, redh, rixh, wraq
NULL, h, q, xzi, zi
	 wrtecci, qkwhi, yowhi, vawwi, jaxsi, whkqhi, ñevvi, sqaryi
NULL, si, sih, siq, sizi
	 ztis, rtis, vyus, ctiq, hqks


In [87]:
lxaCorpus.signatures_to_words()
lxaCorpus.affixes_to_signatures()

{'NULL': {('NULL', 'ci'),
  ('NULL', 'h'),
  ('NULL', 'h', 'i'),
  ('NULL', 'h', 'i', 'iqi'),
  ('NULL', 'h', 'i', 'whi'),
  ('NULL', 'h', 'q'),
  ('NULL', 'h', 'q', 'qxi'),
  ('NULL', 'h', 'q', 'w', 'zi'),
  ('NULL', 'h', 'q', 'x', 'zi'),
  ('NULL', 'h', 'q', 'xzi', 'zi'),
  ('NULL', 'h', 'q', 'zi'),
  ('NULL', 'h', 'w'),
  ('NULL', 'h', 'whi'),
  ('NULL', 'h', 'x'),
  ('NULL', 'h', 'x', 'zi'),
  ('NULL', 'h', 'xi'),
  ('NULL', 'h', 'zi'),
  ('NULL', 'i'),
  ('NULL', 'i', 'ih'),
  ('NULL', 'i', 'ih', 'iq', 'izi'),
  ('NULL', 'i', 'iq'),
  ('NULL', 'i', 'iw'),
  ('NULL', 'i', 'ix'),
  ('NULL', 'i', 'ix', 'ixi'),
  ('NULL', 'i', 'ix', 'ixi', 'ixw'),
  ('NULL', 'i', 'izi'),
  ('NULL', 'i', 'w'),
  ('NULL', 'i', 'ñiz'),
  ('NULL', 'ih'),
  ('NULL', 'ih', 'ixi'),
  ('NULL', 'iq'),
  ('NULL', 'iq', 'iqxi', 'ix'),
  ('NULL', 'iq', 'ix'),
  ('NULL', 'ix'),
  ('NULL', 'ix', 'ixw'),
  ('NULL', 'ix', 'w'),
  ('NULL', 'ixi'),
  ('NULL', 'izi'),
  ('NULL', 'q'),
  ('NULL', 'q', 'qxi'),
  ('NULL', 

In [88]:
lxaCorpus.stems_to_signatures()["zesheq"]

{('NULL', 'ix')}

In [89]:
lxaCorpus.words_in_signatures()
lxaCorpus.words_to_signatures()
lxaCorpus.words_to_sigtransforms()

{'duyzqoxd': {(('NULL', 'ix'), 'NULL')},
 'duyzqoxdix': {(('NULL', 'ix'), 'NULL')},
 'waxzñiz': {(('NULL', 'ix'), 'NULL')},
 'waxzñizix': {(('NULL', 'ix'), 'NULL')},
 'zesheqix': {(('NULL', 'ix'), 'NULL')},
 'zesheq': {(('NULL', 'ix'), 'NULL')},
 'wtexzad': {(('NULL', 'ix'), 'NULL')},
 'wtexzadix': {(('NULL', 'ix'), 'NULL')},
 'teqz': {(('NULL', 'ix'), 'NULL')},
 'teqzix': {(('NULL', 'ix'), 'NULL')},
 'cuqñacciqix': {(('NULL', 'ix'), 'NULL')},
 'cuqñacciq': {(('NULL', 'ix'), 'NULL')},
 'ñayc': {(('NULL', 'ix'), 'NULL')},
 'ñaycix': {(('NULL', 'ix'), 'NULL')},
 'peqsaqy': {(('NULL', 'ix'), 'NULL')},
 'peqsaqyix': {(('NULL', 'ix'), 'NULL')},
 'ñuwjexzix': {(('NULL', 'ix'), 'NULL')},
 'ñuwjexz': {(('NULL', 'ix'), 'NULL')},
 'yityodñizix': {(('NULL', 'ix'), 'NULL')},
 'yityodñiz': {(('NULL', 'ix'), 'NULL')},
 'ytiqiq': {(('NULL', 'ix'), 'NULL')},
 'ytiqiqix': {(('NULL', 'ix'), 'NULL')},
 'cuzziqdqtephix': {(('NULL', 'ix'), 'NULL')},
 'cuzziqdqteph': {(('NULL', 'ix'), 'NULL')},
 'vuxdix': {

## Words manifolds and syntactic word neighborhood
Semblent ne pas fonctionner avec Python3

## Phonology

In [90]:
lxaCorpus.phone_unigram_counter()
lxaCorpus.phone_bigram_counter()
lxaCorpus.phone_trigram_counter()

{('#', '0', '0'): 9,
 ('0', '0', '1'): 1,
 ('0', '1', '#'): 2,
 ('#', 'p', 'q'): 1033,
 ('p', 'q', 't'): 60,
 ('q', 't', 'e'): 1192,
 ('t', 'e', 's'): 85,
 ('e', 's', 'e'): 6,
 ('s', 'e', 'x'): 1478,
 ('e', 'x', 'd'): 1051,
 ('x', 'd', 'i'): 3509,
 ('d', 'i', 'x'): 3615,
 ('i', 'x', '#'): 22724,
 ('#', 'i', 'y'): 337,
 ('i', 'y', 'y'): 564,
 ('y', 'y', 'i'): 3725,
 ('y', 'i', 'q'): 1154,
 ('i', 'q', '#'): 10961,
 ('#', 't', 'i'): 3436,
 ('t', 'i', 'q'): 2800,
 ('i', 'q', 'x'): 916,
 ('q', 'x', 'ñ'): 14,
 ('x', 'ñ', 'i'): 28,
 ('ñ', 'i', 'x'): 1917,
 ('i', 'x', 'q'): 43,
 ('x', 'q', 'o'): 17,
 ('q', 'o', 's'): 129,
 ('o', 's', '#'): 1622,
 ('#', 'o', '#'): 4114,
 ('#', 'd', 'a'): 1468,
 ('d', 'a', 'c'): 494,
 ('a', 'c', 'y'): 457,
 ('c', 'y', 'i'): 549,
 ('y', 'i', '#'): 3944,
 ('#', 'z', 'a'): 3055,
 ('z', 'a', 'd'): 629,
 ('a', 'd', 'i'): 1272,
 ('d', 'i', '#'): 3356,
 ('#', 'z', 'i'): 15251,
 ('z', 'i', 'x'): 7214,
 ('i', 'x', 'd'): 1387,
 ('x', 'd', 'a'): 291,
 ('d', 'a', 'x'): 747,

## Voisinages

Semblent ne pas fonctionner avec Python3